# 패키지 불러오기

In [21]:
from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')
from nltk.tokenize import sent_tokenize

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
import finbert.utils as tools

pd.set_option('max_colwidth', -1)

C:\Users\Quantec\AppData\Local\Temp\ipykernel_11956\3105309101.py:18: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


# 모델 불러오기

In [1]:
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert", cache_dir=None, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

C:\Users\Quantec\AppData\Local\Temp\ipykernel_11956\1789822289.py:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

# 사전 처리한 문장 데이터 불러오기

In [2]:
import pandas as pd
text=pd.read_csv('C:/Users/Quantec/tner-master/ticker_processing_0623.csv')
text=text.iloc[:,2:]
text=text.dropna(subset=['ticker'],how='any')
text.reset_index(inplace=True,drop=True)

text_=list(text.contents)

In [6]:
text

,news_label,contents,entity_list,ticker
0,0,"Kimberly Clark Corp (NYSE: KMB), the third-largest tampon maker, said it had not experienced a product or supply shortage.","Kimberly Clark Corp, KMB",KMB
1,0,Pharmacy chains Walgreens Boots Alliance Inc (NASDAQ: WBA) and CVS Health Corp (NYSE: CVS) said they are experiencing temporary shortages of specific tampon varieties.,"Walgreens Boots Alliance Inc, CVS Health Corp, CVS","WBA, CVS, CVS"
2,0,"As per the WSJ report, tampons from all major brands were available on websites for Target Corporation (NYSE: TGT) and Walmart Inc (NYSE: WMT) for in-store pickup and delivery, though some variants were sold out.","WSJ, Target Corporation, TGT, Walmart Inc, WMT","TGT, TGT, WMT, WMT"
3,0,Availability on Amazon.com (NASDAQ: AMZN) was spotty.,"Amazon.com, AMZN","AMZN, AMZN"
4,2,"Meanwhile, strategists at JPMorgan said the S&P 500's decline implies an 85% chance of recession.",JPMorgan,JPM
...,...,...,...,...
1369,294,"American Airlines plans to drop service to four U.S. cities in September, including Dubuque, Iowa, which will lose scheduled commercial air service altogether.",American Airlines,AAL
1370,294,"American, United Airlines and Delta Air Lines have each scaled back service between some smaller cities and their hubs, citing a lack of aviators.","American, United Airlines, Delta Air Lines","AAL, UAL, DAL"
1371,294,"""We'll proactively reach out to customers scheduled to travel after this date to offer alternate arrangements,"" American said in a statement.",American,AAL
1372,294,The airports were served by American Airlines' regional airline subsidiaries.,American Airlines,AAL


# Predict

In [9]:
result = predict(text_,model,use_gpu=True)

07/11/2022 14:14:05 - INFO - root -   Using device: cuda:0 
07/11/2022 14:14:05 - INFO - finbert.utils -   *** Example ***
07/11/2022 14:14:05 - INFO - finbert.utils -   guid: 0
07/11/2022 14:14:05 - INFO - finbert.utils -   tokens: [CLS] kimberly clark corp ( ny ##se : km ##b ) , the third - largest tam ##pon maker , said it had not experienced a product or supply shortage . [SEP]
07/11/2022 14:14:05 - INFO - finbert.utils -   input_ids: 101 23729 5215 13058 1006 6396 3366 1024 2463 2497 1007 1010 1996 2353 1011 2922 17214 26029 9338 1010 2056 2009 2018 2025 5281 1037 4031 2030 4425 15843 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:05 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:05 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

07/11/2022 14:14:07 - INFO - finbert.utils -   *** Example ***
07/11/2022 14:14:07 - INFO - finbert.utils -   guid: 0
07/11/2022 14:14:07 - INFO - finbert.utils -   tokens: [CLS] after becoming one of the biggest victims of the pan ##de ##mic , amc is seeing a revival in business , driven by a steady stream of new releases such as " doctor strange and the multi ##vers ##ity of madness " and " top gun : maverick " . [SEP]
07/11/2022 14:14:07 - INFO - finbert.utils -   input_ids: 101 2044 3352 2028 1997 1996 5221 5694 1997 1996 6090 3207 7712 1010 21962 2003 3773 1037 6308 1999 2449 1010 5533 2011 1037 6706 5460 1997 2047 7085 2107 2004 1000 3460 4326 1998 1996 4800 14028 3012 1997 12013 1000 1998 1000 2327 3282 1024 27187 1000 1012 102 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:07 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:07 - INFO - finbert.uti

07/11/2022 14:14:07 - INFO - finbert.utils -   *** Example ***
07/11/2022 14:14:07 - INFO - finbert.utils -   guid: 0
07/11/2022 14:14:07 - INFO - finbert.utils -   tokens: [CLS] facing intense competition for talent and pressure to hit diversity goals , banks are getting more creative in their hiring , with ci ##ti ##group inc recruiting u . s . associates this summer without banking experience or business degrees . [SEP]
07/11/2022 14:14:07 - INFO - finbert.utils -   input_ids: 101 5307 6387 2971 2005 5848 1998 3778 2000 2718 8906 3289 1010 5085 2024 2893 2062 5541 1999 2037 14763 1010 2007 25022 3775 17058 4297 14357 1057 1012 1055 1012 9228 2023 2621 2302 8169 3325 2030 2449 5445 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:07 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:07 - INFO - finbert.utils -   token_type_ids: 0 

07/11/2022 14:14:07 - INFO - finbert.utils -   label: None (id = 9090)
07/11/2022 14:14:07 - INFO - root -   tensor([[-0.4506, -1.5367,  2.3319],
        [ 0.2399, -1.9882,  1.8525],
        [ 1.6421, -0.8763, -1.7372],
        [-1.6717,  2.8667, -0.8018],
        [-1.8644,  2.8149, -0.2903]], device='cuda:0')
07/11/2022 14:14:07 - INFO - finbert.utils -   *** Example ***
07/11/2022 14:14:07 - INFO - finbert.utils -   guid: 0
07/11/2022 14:14:07 - INFO - finbert.utils -   tokens: [CLS] they said ryan ##air operated 17 planes out of char ##ler ##oi airport and two out of brussels . [SEP]
07/11/2022 14:14:07 - INFO - finbert.utils -   input_ids: 101 2027 2056 4575 11215 3498 2459 9738 2041 1997 25869 3917 10448 3199 1998 2048 2041 1997 9371 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:07 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

07/11/2022 14:14:07 - INFO - finbert.utils -   input_ids: 101 1000 2392 20660 2015 2005 7044 1010 2009 2003 2664 2178 4603 23299 3029 2008 4071 9316 5826 2007 2000 5083 2256 3277 2275 1010 1000 3580 14191 2072 2056 1999 2019 10373 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:07 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:07 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:07 - INFO - finbert.utils -   label: None (id = 9090)
07/11/2022 14:14:07 - INFO - root -   tensor([[ 1.1888, -2.5932,  0.9336],
        [-1.0892,  0.0337,  1.9083],
        [-0.6351, -1.1865,  2.3390],
        [-0.1002, -2.0947,  2.3200],
        [ 1.8030, -2.3412, -0.0764]], device='cuda:0')
07/11/2022

...

07/11/2022 14:14:17 - INFO - finbert.utils -   label: None (id = 9090)
07/11/2022 14:14:17 - INFO - root -   tensor([[-1.5541,  1.2709,  1.1652],
        [ 0.3745, -2.2785,  1.8638],
        [ 0.3299, -2.4497,  2.0362],
        [-0.9846, -1.0495,  2.4061],
        [-0.9578, -1.0064,  2.4694]], device='cuda:0')
07/11/2022 14:14:17 - INFO - finbert.utils -   *** Example ***
07/11/2022 14:14:17 - INFO - finbert.utils -   guid: 0
07/11/2022 14:14:17 - INFO - finbert.utils -   tokens: [CLS] and if you do that , would you still keep being ceo of tesla and space ##x ? [SEP]
07/11/2022 14:14:17 - INFO - finbert.utils -   input_ids: 101 1998 2065 2017 2079 2008 1010 2052 2017 2145 2562 2108 5766 1997 26060 1998 2686 2595 1029 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:17 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

07/11/2022 14:14:17 - INFO - finbert.utils -   input_ids: 101 1999 2089 1010 14163 6711 3333 3488 2000 6822 4636 2010 5309 1997 10474 2007 1037 7785 5414 5079 2000 2010 26060 8406 1998 3445 1996 2946 1997 1996 3066 1521 1055 10067 6922 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:17 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:17 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:17 - INFO - finbert.utils -   label: None (id = 9090)
07/11/2022 14:14:18 - INFO - root -   tensor([[-0.3162,  0.1221,  0.2163],
        [-0.3056, -1.9219,  2.3295],
        [-0.1654,  1.6214, -1.6672],
        [-1.3909,  2.7264, -1.0738],
        [-1.3166,  2.0751, -0.1926]], device='cuda:0')
07/11/2

07/11/2022 14:14:18 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:18 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:18 - INFO - finbert.utils -   label: None (id = 9090)
07/11/2022 14:14:18 - INFO - root -   tensor([[-1.7521,  3.0284, -0.9671],
        [-1.4969,  2.4142, -0.6915],
        [ 2.0474, -1.7608, -1.5648],
        [ 2.0267, -1.7657, -1.5498],
        [ 2.0615, -1.5214, -1.6738]], device='cuda:0')
07/11/2022 14:14:18 - INFO - finbert.utils -   *** Example ***
07/11/2022 14:14:18 - INFO - finbert.utils -   guid: 0
07/11/2022 14:14:18 - INFO - finbert.utils -   tokens: [CLS] church & dwight ( ny ##se : ch ##d ) stock rose 2 . 4 % after wells fargo upgraded its stance on the household products manufac

07/11/2022 14:14:18 - INFO - finbert.utils -   input_ids: 101 1996 1041 1011 6236 5016 4122 2000 2191 9530 14028 2075 2007 24969 2062 3019 1999 2236 1010 1998 2038 4565 2041 1037 2186 1997 2838 2008 9585 2049 2376 3353 2000 28024 2062 2529 1011 2066 7982 1010 2130 2000 1996 2391 1997 4851 1037 5310 3980 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:18 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:18 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:18 - INFO - finbert.utils -   label: None (id = 9090)
07/11/2022 14:14:18 - INFO - root -   tensor([[ 1.1696, -2.5941,  1.2715],
        [-1.0058, -0.5649,  2.3242],
        [-1.6645,  2.9437, -0.9068],
        [ 1.8308, -2.0754, -1.0223],
        [-1.0124, -0.

07/11/2022 14:14:19 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:19 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/11/2022 14:14:19 - INFO - finbert.utils -   label: None (id = 9090)
07/11/2022 14:14:19 - INFO - root -   tensor([[-1.0604, -0.8565,  2.4378],
        [-1.3890,  2.8112, -1.2866],
        [-1.6724,  2.9508, -1.0279],
        [-1.4015,  1.0529,  0.4184],
        [-1.6120,  2.9509, -1.1363]], device='cuda:0')
07/11/2022 14:14:19 - INFO - finbert.utils -   *** Example ***
07/11/2022 14:14:19 - INFO - finbert.utils -   guid: 0
07/11/2022 14:14:19 - INFO - finbert.utils -   tokens: [CLS] rb ##c also cut earnings estimates on the expectation that the housing market will further deter ##ior ##ate as mortgage rates 

In [10]:
result

,sentence,logit,prediction,sentiment_score
0,"Kimberly Clark Corp (NYSE: KMB), the third-largest tampon maker, said it had not experienced a product or supply shortage.","[0.031951945, 0.94917345, 0.018874606]",negative,-0.917221
1,Pharmacy chains Walgreens Boots Alliance Inc (NASDAQ: WBA) and CVS Health Corp (NYSE: CVS) said they are experiencing temporary shortages of specific tampon varieties.,"[0.01185197, 0.9664528, 0.021695193]",negative,-0.954601
2,"As per the WSJ report, tampons from all major brands were available on websites for Target Corporation (NYSE: TGT) and Walmart Inc (NYSE: WMT) for in-store pickup and delivery, though some variants were sold out.","[0.060072932, 0.041216906, 0.89871013]",neutral,0.018856
3,Availability on Amazon.com (NASDAQ: AMZN) was spotty.,"[0.009881541, 0.96498924, 0.02512922]",negative,-0.955108
4,"Meanwhile, strategists at JPMorgan said the S&P 500's decline implies an 85% chance of recession.","[0.018489862, 0.9707366, 0.010773507]",negative,-0.952247
...,...,...,...,...
1369,"American Airlines plans to drop service to four U.S. cities in September, including Dubuque, Iowa, which will lose scheduled commercial air service altogether.","[0.0065687853, 0.96004444, 0.03338686]",negative,-0.953476
1370,"American, United Airlines and Delta Air Lines have each scaled back service between some smaller cities and their hubs, citing a lack of aviators.","[0.0068113804, 0.9677163, 0.0254724]",negative,-0.960905
1371,"""We'll proactively reach out to customers scheduled to travel after this date to offer alternate arrangements,"" American said in a statement.","[0.8319016, 0.012600542, 0.1554978]",positive,0.819301
1372,The airports were served by American Airlines' regional airline subsidiaries.,"[0.035245493, 0.020900657, 0.94385386]",neutral,0.014345


# 뉴스테마 라벨 Merge

In [11]:
sa_df=result.iloc[:,2:]

In [13]:
last=pd.concat([text,sa_df],axis=1)

In [14]:
test=pd.read_csv('C:/Users/Quantec/quantec/test.csv')

In [15]:
test=test.reset_index()

In [17]:
test_topic=test.loc[:,['index','topics']]

In [18]:
last_label=pd.merge(last,test_topic,left_on='news_label',right_on='index')

In [19]:
last_label.drop('index',axis=1,inplace=True)

In [20]:
last_label

,news_label,contents,entity_list,ticker,prediction,sentiment_score,topics
0,0,"Kimberly Clark Corp (NYSE: KMB), the third-largest tampon maker, said it had not experienced a product or supply shortage.","Kimberly Clark Corp, KMB",KMB,negative,-0.917221,1
1,0,Pharmacy chains Walgreens Boots Alliance Inc (NASDAQ: WBA) and CVS Health Corp (NYSE: CVS) said they are experiencing temporary shortages of specific tampon varieties.,"Walgreens Boots Alliance Inc, CVS Health Corp, CVS","WBA, CVS, CVS",negative,-0.954601,1
2,0,"As per the WSJ report, tampons from all major brands were available on websites for Target Corporation (NYSE: TGT) and Walmart Inc (NYSE: WMT) for in-store pickup and delivery, though some variants were sold out.","WSJ, Target Corporation, TGT, Walmart Inc, WMT","TGT, TGT, WMT, WMT",neutral,0.018856,1
3,0,Availability on Amazon.com (NASDAQ: AMZN) was spotty.,"Amazon.com, AMZN","AMZN, AMZN",negative,-0.955108,1
4,2,"Meanwhile, strategists at JPMorgan said the S&P 500's decline implies an 85% chance of recession.",JPMorgan,JPM,negative,-0.952247,0
...,...,...,...,...,...,...,...
1369,294,"American Airlines plans to drop service to four U.S. cities in September, including Dubuque, Iowa, which will lose scheduled commercial air service altogether.",American Airlines,AAL,negative,-0.953476,8
1370,294,"American, United Airlines and Delta Air Lines have each scaled back service between some smaller cities and their hubs, citing a lack of aviators.","American, United Airlines, Delta Air Lines","AAL, UAL, DAL",negative,-0.960905,8
1371,294,"""We'll proactively reach out to customers scheduled to travel after this date to offer alternate arrangements,"" American said in a statement.",American,AAL,positive,0.819301,8
1372,294,The airports were served by American Airlines' regional airline subsidiaries.,American Airlines,AAL,neutral,0.014345,8


In [22]:
topic_sa_score=pd.DataFrame(last_label.groupby(['topics','ticker'])['sentiment_score'].mean())

In [23]:
topic_sa_count=pd.DataFrame(last_label.groupby(['topics','ticker'])['prediction'].value_counts())

In [70]:
topic_sa_score.to_csv('topic_sa_score.csv',encoding='utf-8-sig')

In [71]:
topic_sa_count.to_csv('topic_sa_count.csv',encoding='utf-8-sig')

In [24]:
test=topic_sa_score.reset_index()

In [29]:
test.set_index('topics')

,ticker,sentiment_score
topics,,
0,"A, AOS",-0.002022
0,AAL,-0.242369
0,AAPL,0.471219
0,"AAPL, AAPL",0.567667
0,"AAPL, AMZN",0.897796
...,...,...
10,"MRNA, MRNA, PFE",0.393553
10,"MRNA, PFE",0.625046
10,PFE,0.416498
